# Russian Synodal Bible (1885)

## Import

### Libraries

In [1]:
import os
from lxml import etree
from bs4 import BeautifulSoup

from backend import *

In [2]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)

### Definitions

In [3]:
bibleTXT = './texts/bible/sinodalnyi-perevod.txt' 
booksJSON = './texts/bible/booksDict.json'
bibleJSON = './texts/bible/bible.json'
bibleIdJSON = './texts/bible/bibleID.json'
bibleXML = './texts/bible/bible.xml'

bibleOHCO = ['test', 'book', 'chap', 'verse']
tokenCols = ['p_id', 'start', 'stop', 'text', 'token_id', 'head_id', 'rel', 'pos', 'lemma', 'anim', 'aspect', 'case', 'degree', 'gender', 'mood', 'number', 'person', 'tense', 'verb_form', 'voice']

chap_lines_re = '^===\s(\d{1,3})\s===$'
book_lines_re = '^==\s(.+)\s==$'
matt_name = 'От Матфея святое благовествование'

In [4]:
with open(booksJSON) as json_file: 
    booksDict = json.load(json_file)

## Pre-Processing

### Import Text into DF

### Tidy Up

### Find Parts

### Assign OHCO Labels

### Swap Full Book Name for Abbreviation

### Export to JSON

## Processing

In [5]:
bibliiaDf = pd.read_json(bibleJSON, orient='index')
bibliiaDf.index.name = 'v_id'
#BibTextDf = bibliiaDf[['text']]
#BibLibDf = bibliiaDf[bibleOHCO]
bibliiaDf

,test,book,chap,verse,text
v_id,,,,,
1,O,Gen,1,1,В начале сотворил Бог небо и землю.
2,O,Gen,1,2,"Земля же была безвидна и пуста, и тьма над без..."
3,O,Gen,1,3,И сказал Бог: да будет свет. И стал свет.
4,O,Gen,1,4,"И увидел Бог свет, что он хорош, и отделил Бог..."
5,O,Gen,1,5,"И назвал Бог свет днем, а тьму ночью. И был ве..."
...,...,...,...,...,...
37106,N,Rev,22,17,И Дух и невеста говорят: прииди! И слышавший д...
37107,N,Rev,22,18,И я также свидетельствую всякому слышащему сло...
37108,N,Rev,22,19,и если кто отнимет что от слов книги пророчест...


### OHCO DFs

In [6]:
#testsDict = dict(enumerate(bibliiaDf.test.unique()))
testsDict = dict([(value, key) for key, value in dict(enumerate(bibliiaDf.test.unique())).items()])
booksDict = dict([(value, key) for key, value in dict(enumerate(bibliiaDf.book.unique())).items()])

#### Testaments

In [7]:
TestsDf = pd.DataFrame([(x, ' '.join(y)) for (x,y) in bibliiaDf.groupby(bibleOHCO[:1]).text], columns=['test', 'text']).set_index(np.arange(1,len(bibliiaDf.test.unique())+1))
TestsDf.index.name = 't_id'
#TestsDf 

In [8]:
bibliiaDf['t_id'] = bibliiaDf.test.map(testsDict)

#### Books

In [9]:
BooksDf = pd.DataFrame([(x, ' '.join(y)) for (x,y) in bibliiaDf.groupby(bibleOHCO[:2]).text], columns=[('test', 'book'), 'text'])
BooksDf[['test','book']] = pd.DataFrame(list(BooksDf[('test', 'book')]), index=BooksDf.index, columns=bibleOHCO[:2])
del BooksDf[('test', 'book')]
BooksDf = BooksDf.replace({"test": testsDict, "book": booksDict}).sort_values(by=bibleOHCO[:2], ascending=[True, True])
BooksDf = BooksDf.reset_index().drop(['index'], axis=1).set_index(np.arange(1,len(bibliiaDf.book.unique())+1))
BooksDf.index.name = 'b_id'
BooksDf = BooksDf.replace({"test":dict(enumerate(bibliiaDf.test.unique())), "book":dict(enumerate(bibliiaDf.book.unique()))})
#BooksDf = BooksDf.reset_index().set_index(bibleOHCO[:2])
#BooksDf 

In [10]:
bibliiaDf['b_id'] = bibliiaDf.book.map(BooksDf.reset_index().set_index('book')['b_id'])

#### Chapters

In [11]:
ChapsDf = pd.DataFrame([(x, ' '.join(y)) for (x,y) in bibliiaDf.groupby(bibleOHCO[:3]).text], columns=[('test', 'book', 'chap'), 'text'])
ChapsDf[['test','book','chap']] = pd.DataFrame(list(ChapsDf[('test', 'book', 'chap')]), index=ChapsDf.index, columns=bibleOHCO[:3])
del ChapsDf[('test', 'book', 'chap')]
ChapsDf = ChapsDf.replace({"test": testsDict, "book": booksDict}).sort_values(by=bibleOHCO[:3], ascending=[True, True, True])
ChapsDf = ChapsDf.reset_index().drop(['index'], axis=1).set_index(np.arange(1,len(ChapsDf)+1))
ChapsDf.index.name = 'c_id'
ChapsDf = ChapsDf.replace({"test":dict(enumerate(bibliiaDf.test.unique())), "book":dict(enumerate(bibliiaDf.book.unique()))})
#ChapsDf = ChapsDf.reset_index().set_index(bibleOHCO[:3])
#ChapsDf

In [12]:
bibliiaDf = pd.merge(bibliiaDf, ChapsDf.reset_index()[['book', 'chap', 'c_id']], on=['book', 'chap'], how='inner')
bibliiaDf['v_id'] = bibliiaDf.index.to_series().apply(lambda x: x+1)
bibliiaDf = bibliiaDf.set_index('v_id')[['t_id', 'b_id', 'c_id', 'text', 'test', 'book', 'chap']]
bibliiaDf

,t_id,b_id,c_id,text,test,book,chap
v_id,,,,,,,
1,0,1,1,В начале сотворил Бог небо и землю.,O,Gen,1
2,0,1,1,"Земля же была безвидна и пуста, и тьма над без...",O,Gen,1
3,0,1,1,И сказал Бог: да будет свет. И стал свет.,O,Gen,1
4,0,1,1,"И увидел Бог свет, что он хорош, и отделил Бог...",O,Gen,1
5,0,1,1,"И назвал Бог свет днем, а тьму ночью. И был ве...",O,Gen,1
...,...,...,...,...,...,...,...
37106,1,76,1361,И Дух и невеста говорят: прииди! И слышавший д...,N,Rev,22
37107,1,76,1361,И я также свидетельствую всякому слышащему сло...,N,Rev,22
37108,1,76,1361,и если кто отнимет что от слов книги пророчест...,N,Rev,22


In [13]:
bibliiaDf.to_json(bibleIdJSON, orient='index')

### Make XML

In [ ]:
bibliiaDf.reset_index().set_index(bibleOHCO[:4])

In [ ]:

#bibleTree = etree.ElementTree()

In [ ]:
root = etree.Element("bible")
print(root.tag)
TestList = [x for x in bibliiaDf.test.unique()]
BookList = [x for x in bibliiaDf.book.unique()]
for test in range(len(TestList)): 
    root.append(etree.Element("t", name=TestList[test]))
    #for book in list(bibliiaDf.loc[bibliiaDf.test == test].book.unique()): 
        #test.append(etree.Element("b", name=book))
etree.tostring(root, pretty_print=True)

In [ ]:
root[0].attrib

In [ ]:
for test in range(len(TestList)): 
    print(f"{TestList[test]}")

In [ ]:
bibliiaDf

In [ ]:
root.append(etree.Element("testament"))

In [ ]:
root.append(etree.Element(""))

In [ ]:
etree.tostring(root, pretty_print=True)

## Token DFs

In [ ]:
%%time
ChapsTokenDf = nat_parse(ChapsDf)

In [ ]:
%%time
VerseTokenDf = nat_parse(BibTextDf)

In [ ]:
VerseTokenDf

In [ ]:
VerseTokenDf.to_pickle('./proc/BibleVerseTokenDf.pkl')

In [ ]:
tokenDf = pd.read_pickle('./proc/BibleTokenDf.pkl')
tokenDf.set_index(['v_id', 'token_id'])

In [ ]:
tokenDf.query("pos == 'PRON'").lemma.unique()

In [ ]:
ChapsTokenDf.set_index(['p_id', 'token_id'])

In [ ]:
ChapsTokenDf.query("pos == 'PRON'").lemma.unique()

In [ ]:
%%time
TokenDf = nat_parse(BibTextDf).set_index(['p_id', 'token_num'])

In [ ]:
TokenDf.to_pickle('./proc/BibleTokenDf.pkl')